In [2]:
import os
from os import listdir,makedirs
from os.path import isfile,join
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt
import tensorflow as tf
import keras.backend as K
import keras
from keras.models import Sequential
import cv2



C:\Users\Meers\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [26]:
'''
Hyper-Parameters
'''
image_H = 96.0
image_W = 96.0
grayscale = True
batch_size = 16
learning_rate=0.001
momentum = 0
decay = 0
epochs = 100
droupout= 0.1
gradient_clipping = 0.1
nFilters = 128
nlandmarks = nLabels = 68
K=45
img_rows = 96
img_cols = 96
img_channels = 1

In [54]:
# Get files
import pts_loader
img_x = []
pts_y = []
for i in range(1,50):
    if i < 10:
        img_f = r"C:\Users\Meers\Documents\Techie\InkersIntern\Phase2\FLDCode\300W\300w\01_Indoor\indoor_00" + str(i) +".png"
        pts_f = r"C:\Users\Meers\Documents\Techie\InkersIntern\Phase2\FLDCode\300W\300w\01_Indoor\indoor_00" + str(i) +".pts"
    elif i < 100:
        img_f = r"C:\Users\Meers\Documents\Techie\InkersIntern\Phase2\FLDCode\300W\300w\01_Indoor\indoor_0" + str(i) +".png"
        pts_f = r"C:\Users\Meers\Documents\Techie\InkersIntern\Phase2\FLDCode\300W\300w\01_Indoor\indoor_0" + str(i) +".pts"
    else:
        img_f = r"C:\Users\Meers\Documents\Techie\InkersIntern\Phase2\FLDCode\300W\300w\01_Indoor\indoor_" + str(i) +".png"
        pts_f = r"C:\Users\Meers\Documents\Techie\InkersIntern\Phase2\FLDCode\300W\300w\01_Indoor\indoor_" + str(i) +".pts"
    pts = pts_loader.load(pts_f)
    # Loading imgae in grayscale
    img = cv2.resize(cv2.imread(img_f), (img_rows, img_cols))
    img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    img = img.reshape(img_rows, img_cols, 1)
    scale_x = image_W/img.shape[0]
    scale_y = image_H/img.shape[1]
    new_points =[]
    for p in pts:
        new_pts = []
        new_pts = (p[0] * scale_y , p[1] * scale_x )
        new_points.append(new_pts)
    img_x.append(img)
    pts_y.append(new_points)


In [55]:
print(img_x[0].shape)

train_x = []
train_y = []
test_x = []
test_y = []


(96, 96, 1)


In [62]:
train_x = np.array(img_x[:32])
train_y = np.array(pts_y[:32])
test_x = np.array(img_x[32:])
test_y = np.array(pts_y[32:])


In [63]:
f = open(r'C:\Users\Meers\Documents\Techie\InkersIntern\Phase2\Merget\Merget\network\config\kernel45.txt')
kernel45 = np.fromfile(f, dtype = np.float32, count = -1, sep = " ")

In [64]:
from keras.layers import Dense, Dropout, Activation, Add
from keras.layers import Convolution2D, Lambda, Input, Reshape
from keras.layers.normalization import BatchNormalization
from keras.models import Model

def apply_kernel(local_out):
    '''
     get kernel from text file
     tile it to 1x1x45x45
     convolve with local out
    '''
    f = open(r'C:\Users\Meers\Documents\Techie\InkersIntern\Phase2\Merget\Merget\network\config\kernel45.txt')
    kernel45 = np.fromfile(f, dtype = np.float32, count = -1, sep = " ")
    
    # [1 x 1 x 45 x 45]
    kernel45 = np.reshape(kernel45, (45, 45, 1, 1))
    # [numLabels x 1 x 45 x 45] if tensor convolution is used
    kernel_stacked 	= np.tile(kernel45, (1, 1, nLabels,1))
    #heatmaps = Convolution2D(local_out, kernel_stacked)
    kernel_tensor = tf.convert_to_tensor(kernel_stacked, dtype=tf.float32)
    print("local_out", local_out)
    print("kernel", kernel_tensor)
    '''
    May be use this for group Convolution
    import tensorlayer as tl
    tl.layers.GroupConv2d(prev_layer, n_filter=32, filter_size=(3, 3), strides=(2, 2), n_group=2, act=None,
    padding='SAME', W_init=<tensorflow.python.ops.init_ops.TruncatedNormal object>, 
    b_init=<tensorflow.python.ops.init_ops.Constant object>, W_init_args=None, b_init_args=None, name='groupconv')
    '''
    return tf.nn.conv2d(local_out, kernel_tensor, strides=[1,1,1,1], padding="SAME")
    #return heatmaps


'''
LOCAL SUBNET
'''

# [ImageC x ImageH x ImageW]

input_img = Input(( img_rows, img_cols, img_channels))
model = Convolution2D(filters=nFilters, kernel_size=(5, 5), padding='same', activation='relu')(input_img)
model = BatchNormalization()(model)
model = Convolution2D(filters=nFilters, kernel_size=(5, 5), padding='same', activation='relu')(model)
model = BatchNormalization()(model)
model = Convolution2D(filters=nFilters, kernel_size=(5, 5), padding='same', activation='relu')(model)
model = BatchNormalization()(model)
model = Convolution2D(filters=nFilters, kernel_size=(5, 5), padding='same', activation='relu')(model)
model = BatchNormalization()(model)
model = Convolution2D(filters=nFilters, kernel_size=(5, 5), padding='same', activation='relu')(model)
model = BatchNormalization()(model)

model = Convolution2D(filters=nFilters, kernel_size=(3, 3), padding='same', activation='relu')(model)
model = BatchNormalization()(model)
model = Convolution2D(filters=nFilters, kernel_size=(3, 3), padding='same', activation='relu')(model)
model = BatchNormalization()(model)
model = Convolution2D(filters=nFilters, kernel_size=(3, 3), padding='same', activation='relu')(model)
model = BatchNormalization()(model)
model = Convolution2D(filters=nFilters, kernel_size=(3, 3), padding='same', activation='relu')(model)
model = BatchNormalization()(model)
model = Convolution2D(filters=nFilters, kernel_size=(3, 3), padding='same', activation='relu')(model)
model = BatchNormalization()(model)
model = Convolution2D(filters=nFilters, kernel_size=(3, 3), padding='same', activation='relu')(model)
model = BatchNormalization()(model)
model = Convolution2D(filters=nFilters, kernel_size=(3, 3), padding='same', activation='relu')(model)
model = BatchNormalization()(model)
model = Convolution2D(filters=nFilters, kernel_size=(3, 3), padding='same', activation='relu')(model)
model = BatchNormalization()(model)
model = Convolution2D(filters=nFilters, kernel_size=(3, 3), padding='same', activation='relu')(model)
model = BatchNormalization()(model)
model = Convolution2D(filters=nFilters, kernel_size=(3, 3), padding='same', activation='relu')(model)
model = BatchNormalization()(model)
print("Local subnet", model)
'''
1x1 Convolution with activation
'''
model_Conv = Convolution2D(filters=nLabels, kernel_size=(1, 1),activation='relu')(model)
model_Conv = Lambda(apply_kernel, output_shape=(nLabels, img_cols, img_rows))(model_Conv)
print("Main 1x1", model_Conv)
'''
Ol 1x1 Linear convolution
Apply Gaussian Kernel filter
'''
model_Ol = Convolution2D(filters=nLabels, kernel_size=(1, 1),activation=None)(model)
# What about tiling the kernels????
model_Ol = Lambda(apply_kernel, output_shape=(nLabels, img_cols, img_rows), name='Ol')(model_Ol)
print("Local output", model_Ol)

output_Ol = keras.layers.concatenate([model_Conv, model_Ol])
print("Concatenate", output_Ol)
output_Ol = Reshape((96, 96,-1))(output_Ol)
print("Reshape", output_Ol)

'''
GLOBAL SUBNET
'''
model = Convolution2D(filters=nFilters, kernel_size=(3, 3), padding='same', dilation_rate = 4, activation='relu')(output_Ol)
model = BatchNormalization()(model)
model = Convolution2D(filters=nFilters, kernel_size=(3, 3), padding='same', dilation_rate = 4, activation='relu')(model)
model = BatchNormalization()(model)
model = Convolution2D(filters=nFilters, kernel_size=(3, 3), padding='same', dilation_rate = 4, activation='relu')(model)
model = BatchNormalization()(model)
model = Convolution2D(filters=nFilters, kernel_size=(3, 3), padding='same', dilation_rate = 4, activation='relu')(model)
model = BatchNormalization()(model)
model = Convolution2D(filters=nFilters, kernel_size=(3, 3), padding='same', dilation_rate = 4, activation='relu')(model)
model = BatchNormalization()(model)
model = Convolution2D(filters=nFilters, kernel_size=(3, 3), padding='same', dilation_rate = 4, activation='relu')(model)
model = BatchNormalization()(model)
model = Convolution2D(filters=nFilters, kernel_size=(3, 3), padding='same', dilation_rate = 4, activation='relu')(model)
model = BatchNormalization()(model)
model = Convolution2D(filters=nLabels, kernel_size=(1, 1), activation=None)(model)
FLD_model = Model(inputs = input_img, outputs=model)
FLD_model.summary()


Local subnet Tensor("batch_normalization_147/cond/Merge:0", shape=(?, 96, 96, 128), dtype=float32)
local_out Tensor("conv2d_166/Relu:0", shape=(?, 96, 96, 68), dtype=float32)
kernel Tensor("lambda_7/Const:0", shape=(45, 45, 68, 1), dtype=float32)
Main 1x1 Tensor("lambda_7/Conv2D:0", shape=(?, 96, 96, 1), dtype=float32)
local_out Tensor("conv2d_167/BiasAdd:0", shape=(?, 96, 96, 68), dtype=float32)
kernel Tensor("Ol_6/Const:0", shape=(45, 45, 68, 1), dtype=float32)
Local output Tensor("Ol_6/Conv2D:0", shape=(?, 96, 96, 1), dtype=float32)
Concatenate Tensor("concatenate_7/concat:0", shape=(?, 96, 96, 2), dtype=float32)
Reshape Tensor("reshape_7/Reshape:0", shape=(?, 96, 96, 136), dtype=float32)
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_7 (InputLayer)             (None, 96, 96, 1)     0                                         

* Loss Function: Weighted Square Error
* Exclude non-labeled landmarks from the loss
* Weight each of N landmarks depending on whether they lie within the image boundary:
\begin{equation*}
wij = V (i,j) + \frac{L(i,j)}{10} + \frac{L(i,j)}{2N}\sum_{k=1}^N V (i,k),       \\[1pt]
\text{    where V (i; j) and L(i; j) are indicator functions with L(i; j) = 1 iff landmark j in face i is labeled}    \\[1pt]
\text {V (i; j) = 1 iff landmark j in face i is within the image boundary, zero otherwise.    }\\[1pt]
\text{Note that L(i; j) = 0 implies V (i; j) = 0.}\end{equation*}

\begin{equation*}
e_i = \sum_{n=1}^N w_{in}(((O^l_n - G_n) * K)^2 + (O^g_n - G_n * K)^2), \\[2pt]
\text{    where    } \\[2pt]
O^l_n    and    O^g_n = \text{nth channels of the local- and global context subnet outputs, respectively.} \\[1pt]
\text{K = kernel }   \\[1pt]
\text{Gn = ground truth heatmap for landmark n }   \\[1pt]
\text{For efficiency, the kernel convolution after stacking is reused for the loss}
\end{equation*}


In [65]:
import tensorflow as tf
from keras import backend as K

def landmark_weight(labels):
    #[numLabels x 1 x 1]
    Win = tf.reduce_mean(labels)
    Win = tf.multiply(Win, 0.5)
    return Win

'''
Ol - [numLabels x ImageH x ImageW] Kernel conv already applied in Network
Og - [numLabels x ImageH x ImageW]
labels - Landmark coordinates [numLabels x ImageH x ImageW] without heatmaps
'''
def loss_function(Og, labels):#Ol, Og,labels):
    
    # Gn = labels x K
    Gn = apply_kernel(labels)

    Win = landmark_weight(labels)
    
    # Og - Gn
    global_diff = tf.subtract(Og, Gn)
    global_err = tf.square(global_diff)
    
    # Ol - Gn
    '''
    Ol = FLD_model.get_layer(name='Ol')...
    How to access intermediate layer or model from here???
    Currently accessing since Ol is global...
    '''
    local_diff = tf.subtract(model_Ol, Gn)
    local_err = tf.square(local_diff)
    
    comb_err = tf.add(local_err, global_err)
    
    wt_square_err = tf.reduce_sum(Win * comb_err)
    return wt_square_err    


In [66]:
FLD_model.compile(optimizer='sgd',
              loss=loss_function,
              metrics=['accuracy'])


local_out Tensor("conv2d_175/BiasAdd:0", shape=(?, 96, 96, 68), dtype=float32)
kernel Tensor("loss_5/conv2d_175_loss/Const:0", shape=(45, 45, 68, 1), dtype=float32)


In [67]:
 FLD_model.fit(train_x, train_y, batch_size=8, epochs=1, validation_data=(test_x, test_y))

Train on 32 samples, validate on 17 samples
Epoch 1/1


ValueError: Cannot feed value of shape (8, 68, 2) for Tensor 'conv2d_175_target:0', which has shape '(?, ?, ?, ?)'

In [38]:
import cntk as C

cntk_model = C.load_model(r"C:\Users\Meers\Documents\Techie\InkersIntern\Phase2\Merget\Merget\python\output\models\model.21")

In [39]:
cntk_model.parameters

(Parameter('W', [], [68 x 128 x 1 x 1]),
 Parameter('b', [], [68 x 1 x 1]),
 Parameter('scale', [], [128]),
 Parameter('bias', [], [128]),
 Parameter('W', [], [128 x 128 x 3 x 3]),
 Parameter('b', [], [128 x 1 x 1]),
 Parameter('scale', [], [128]),
 Parameter('bias', [], [128]),
 Parameter('W', [], [128 x 128 x 3 x 3]),
 Parameter('b', [], [128 x 1 x 1]),
 Parameter('scale', [], [128]),
 Parameter('bias', [], [128]),
 Parameter('W', [], [128 x 128 x 3 x 3]),
 Parameter('b', [], [128 x 1 x 1]),
 Parameter('scale', [], [128]),
 Parameter('bias', [], [128]),
 Parameter('W', [], [128 x 128 x 3 x 3]),
 Parameter('b', [], [128 x 1 x 1]),
 Parameter('scale', [], [128]),
 Parameter('bias', [], [128]),
 Parameter('W', [], [128 x 128 x 3 x 3]),
 Parameter('b', [], [128 x 1 x 1]),
 Parameter('scale', [], [128]),
 Parameter('bias', [], [128]),
 Parameter('W', [], [128 x 128 x 3 x 3]),
 Parameter('b', [], [128 x 1 x 1]),
 Parameter('scale', [], [128]),
 Parameter('bias', [], [128]),
 Parameter('W',

In [42]:
cntk_model.save("cntk_model.h5")

In [47]:
FLD = keras.models.load_model(r"C:\Users\Meers\Documents\Techie\InkersIntern\Phase2\FLDCode\cntk_model.h5")

OSError: Unable to open file (File signature not found)

In [48]:
import keras.backend as K

#save_model_hdf5(cntk_model, 'my_model.h5')
kerasmodel = K.functional_ops. load_model(cntk_model)

AttributeError: module 'keras.backend' has no attribute 'load_model'

In [21]:
#func = cntk_model.outputs[0]
cntk_model.save('cntk_model.onnx', format=C.ModelFormat.)

In [24]:
import onnx
onnx_model = onnx.load('cntk_model.onnx')

In [26]:
from onnx_tf.backend import prepare
tf_model = prepare(onnx_model)

ValidationError: Required attribute 'consumed_inputs' is missing.

==> Context: Bad node spec: input: "Plus5153_Output_0" input: "Parameter106" input: "Parameter107" input: "Constant108" input: "Constant109" output: "BatchNormalization5165_Output_0" name: "BatchNormalization5165" op_type: "BatchNormalization" attribute { name: "spatial" i: 1 type: INT } attribute { name: "is_test" i: 1 type: INT } attribute { name: "epsilon" f: 1e-05 type: FLOAT } attribute { name: "momentum" f: 0 type: FLOAT } doc_string: "" domain: ""